# Warehouse Problem - Lagrangian relaxation

Consider the MIP formulation:

\begin{align*}
\min &\qquad \sum_w c_w x_w + \sum_{w ,c} t_{w,c} y_{w,c} & \\
\text{subject to:} &&\\
&y_{w,c} \leq x_w & \forall w,c \\
&\sum_w y_{w,c} = 1 & \forall c \\
&\sum_c y_{w,c} \leq C_w  & \forall w \\
&x_w, y_{w,c}, z_c \in \mathbb{B} & \forall w,c
\end{align*}

In [1]:
fixed = 30  # c_w (all the costs are the same)
M = 10*fixed  # the penalty
capacity = [1,4,2,1,3]  # the capacity of a warehouse
supplyCost = [[20,24,11,25,30],  # t_{w,c}
              [28,27,82,83,74],
              [74,97,71,96,70],
              [2,55,73,69,61],
              [46,96,59,83,4],
              [42,22,29,67,59],
              [1,5,73,59,56],
              [10,73,13,43,96],
              [93,35,63,85,46],
              [47,65,55,71,95]]
nbStores = len(supplyCost)
nbWarehouses = len(capacity)
Stores = range(nbStores)
Warehouses = range(nbWarehouses)
UB = nbWarehouses * fixed + sum(max(supplyCost[s][w] for w in Warehouses) for s in Stores)

### Excercices

#### 1. Find the lagrangian relaxation formulation where the constraints 3 are relaxed.

\begin{align*}
\max_{\lambda_c} ~ \min \qquad \sum_w c_w x_w +& \sum_{w ,c} t_{w,c} y_{w,c} + \sum_c \lambda_c (1 - \sum_w y_{w,c}) & \\
\text{subject to:} \\
y_{w,c} \leq x_w && \forall w,c \\
\sum_w y_{w,c} = 1 & & \forall c \\
\sum_c y_{w,c} \leq C_w  & \forall w \\
x_w, y_{w,c}, z_c \in \mathbb{B} && \forall w,c
\end{align*}

#### 2. Implement the lagrangian relaxation.

In [2]:
from docplex.mp.model import Model

def lagrangian_relaxation(rlambda=[fixed]*nbStores):
    mdl = Model(name='Lagrangian')
    x = mdl.binary_var_list(nbWarehouses,name='open')
    y = mdl.binary_var_matrix(nbStores, nbWarehouses, name='supplier')
    
    for w in Warehouses:
        for s in Stores:
            mdl.add_constraint(y[s,w] <= x[w], 'supply_%d_%d' % (s, w))
    for w in Warehouses:
        mdl.add_constraint(sum(y[s,w] for s in Stores) <= capacity[w] * x[w], 'open_%d' % w)
    
    mdl.minimize(fixed * sum(x) + sum(supplyCost[s][w] * y[s,w]  for s in Stores for w in Warehouses)
                 + sum(rlambda[s] * (1 - sum(y[s,w] for w in Warehouses)) for s in Stores))
    
    
    #             
   
    msol = mdl.solve()
    relaxation = [max(0,(1 - sum(y[s,w].solution_value for w in Warehouses))) for s in Stores]
    
    return (mdl.objective_value,  # the objective value
            msol, # solution
            relaxation)  # slack of the relaxed constraints

#### Try the function

In [3]:
obj, msol, relaxed = lagrangian_relaxation()
print(msol, relaxed)

solution for: Lagrangian
objective: 282
open_1=1
open_2=1
supplier_0_1=1
supplier_0_2=1
supplier_1_1=1
supplier_5_1=1
supplier_6_1=1
supplier_7_2=1
 [0, 0, 1, 1, 1, 0, 0, 0, 1, 1]


#### 3. Find a feasible solution

Update the lagrangian coefficients until you get a feasible solution.
While you haven't obtained a feasible solution, do:
1. Compute the lagrangian relaxation
2. Check if feasible. If yes, stop.
3. Update the lagrangian coeeficients (use the gradient of the lagrangian relaxation).

In [7]:
rlambda = [fixed]*nbStores
fsol = None
iteration = 0
step = 2.0 
LB = 0

while True:

    # 1. Compute the lagrangian relaxation
    obj, fsol, relaxation = lagrangian_relaxation(rlambda)
    if obj > LB:
        LB = obj

        # 2. Check if feasible: all slack should be null
    n_violations = sum(int(abs(s)) for s in relaxation)
    
    print('iteration={:d}, obj={:.2f}, n violations={:d}'.format(iteration, obj, n_violations))
    if n_violations == 0:
        break
    iteration+= 1
    
#    nv2 = sum(int(abs(s)*abs(s)) for s in relaxation)
#   step = 0.001 * (bestFeasibleValue - obj) / nv2
#    print('step: ' + str(step))
 
# 3. Update the lagrangian
    for i, s in enumerate(relaxation):
        rlambda[i] += s * step
        if rlambda[i] < 0:
            rlambda[i] = 0
        

#print('========================')
#print(fsol)

iteration=0, obj=282.00, n violations=5
iteration=1, obj=292.00, n violations=5
iteration=2, obj=300.00, n violations=4
iteration=3, obj=306.00, n violations=3
iteration=4, obj=312.00, n violations=3
iteration=5, obj=316.00, n violations=3
iteration=6, obj=322.00, n violations=3
iteration=7, obj=326.00, n violations=3
iteration=8, obj=331.00, n violations=2
iteration=9, obj=335.00, n violations=2
iteration=10, obj=339.00, n violations=2
iteration=11, obj=343.00, n violations=2
iteration=12, obj=347.00, n violations=2
iteration=13, obj=351.00, n violations=2
iteration=14, obj=355.00, n violations=2
iteration=15, obj=359.00, n violations=2
iteration=16, obj=363.00, n violations=2
iteration=17, obj=367.00, n violations=2
iteration=18, obj=371.00, n violations=2
iteration=19, obj=375.00, n violations=2
iteration=20, obj=379.00, n violations=2
iteration=21, obj=381.00, n violations=1
iteration=22, obj=381.00, n violations=1
iteration=23, obj=381.00, n violations=2
iteration=24, obj=379.00, 